In [304]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [305]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'

file_beg = '../NHANES-Downloader/data/csv_data/'

In [306]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [307]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [308]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [309]:
smqfam_indx = [95, 112, 126, 122, 121, 128, 132, 161, 127, 184]

In [310]:
dfs = defaultdict(int)
for i in range(0,len(smqfam_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][smqfam_indx[i]])

## Functions: Recategorize values, Count Values, Drop Rows

In [311]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Important Variables

In [312]:
#1999-2012
var_names = ["SEQN", "SMD410"]

#2013-2018
var_names1 = ["SEQN", "SMD460"]

In [313]:
#To display all columns in Jupyter Notebooks
#pd.set_option('display.max_columns', 500)

## Round the column

In [314]:
dfs[0]['SMD410'] = dfs[0]['SMD410'].round(0)
dfs[1]['SMD410'] = dfs[1]['SMD410'].round(0)
dfs[2]['SMD410'] = dfs[2]['SMD410'].round(0)
dfs[3]['SMD410'] = dfs[3]['SMD410'].round(0)
dfs[4]['SMD410'] = dfs[4]['SMD410'].round(0)
dfs[5]['SMD410'] = dfs[5]['SMD410'].round(0)
dfs[6]['SMD410'] = dfs[6]['SMD410'].round(0)
dfs[7]['SMD460'] = dfs[7]['SMD460'].round(0)
dfs[8]['SMD460'] = dfs[8]['SMD460'].round(0)
dfs[9]['SMD460'] = dfs[9]['SMD460'].round(0)

## Make a copy of the dataframes

In [315]:
#1999-2018
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names].copy() #2003-2004
df3 = dfs[3][var_names].copy() #2005-2006
df4 = dfs[4][var_names].copy() #2007-2008
df5 = dfs[5][var_names].copy() #2009-2010
df6 = dfs[6][var_names].copy() #2011-2012
df7 = dfs[7][var_names1].copy() #2013-2014
df8 = dfs[8][var_names1].copy() #2015-2016
df9 = dfs[9][var_names1].copy() #2017-2018

## Recategorize values

In [316]:
#Recategorize SMD460 to SMD410 -> {2,3 to 1}, 
#{0 to 2}, {777 to 7}, {999 to 9}

#2013-2014
recategorize(df7, 'SMD460', {2:1, 3:1, 0:2, 777:7, 999:9})

#2015-2016
recategorize(df8, 'SMD460', {2:1, 3:1, 0:2, 777:7, 999:9})

## Rename columns 2013 - 2018

In [317]:
#Col names
col_names = var_names

In [318]:
#2013 - 2018
df7.columns = col_names
df8.columns = col_names
df9.columns = col_names

## Append years 1999 - 2018

In [319]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016","2017-2018"]

In [320]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [321]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [322]:
result_1999_2018 = pd.concat(frames, keys = years)

In [323]:
result_1999_2018_cleaned = result_1999_2018.copy()

## Count values for each column

In [324]:
result_1999_2018_cleaned

SEQN  SMD410       Year
1999-2000 0          1.0     2.0  1999-2000
          1          2.0     2.0  1999-2000
          2          3.0     1.0  1999-2000
          3          4.0     2.0  1999-2000
          4          5.0     2.0  1999-2000
...                  ...     ...        ...
2017-2018 9249  102952.0     0.0  2017-2018
          9250  102953.0     0.0  2017-2018
          9251  102954.0     0.0  2017-2018
          9252  102955.0     2.0  2017-2018
          9253  102956.0     1.0  2017-2018

[101316 rows x 3 columns]

In [325]:
len(col_names)

2

In [326]:
#SMD410 - Does anyone smoke in the home
count_vals(result_1999_2018_cleaned, col_names[1])

SMD410
0.0       6092
1.0      19513
2.0      73846
3.0        153
7.0         22
9.0         15
777.0        2
999.0       10
Name: SEQN, dtype: int64 

 NaN:  1663


In [327]:
result_1999_2018[:1]

,,SEQN,SMD410,Year
1999-2000,0,1.0,2.0,1999-2000


In [328]:
len(result_1999_2018)

101316

## Remove missing values:

In [329]:
drop_rows(result_1999_2018_cleaned, col_names[1], [np.nan, 7, 9])

## See if missing values have been correctly removed:

In [330]:
count_vals(result_1999_2018_cleaned, col_names[1])

SMD410
0.0       6092
1.0      19513
2.0      73846
3.0        153
777.0        2
999.0       10
Name: SEQN, dtype: int64 

 NaN:  0


In [331]:
before = len(result_1999_2018)
before

101316

In [332]:
after = len(result_1999_2018_cleaned)
after

99616

In [333]:
(before-after)/after

0.017065531641503373

## MongoDB Insertion

In [334]:
#Import MongoClient
from pymongo import MongoClient

#Create a MongoClient to run the MongoDB instance
client = MongoClient('localhost', 27017)

In [335]:
#Connect to existing datbase
db = client.NHANES

In [336]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [337]:
db.list_collection_names()

['hiq',
 'alq',
 'smqfam',
 'drxtot',
 'smq',
 'bpq',
 'bmx',
 'huq',
 'bpx',
 'mcq_h',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'demo',
 'diq',
 'mcq_c']

In [338]:
#Create collection in database
smqfam = db.smqfam

In [339]:
#If collections exist, then drop
if 'smqfam' in db.list_collection_names():
    smqfam.drop()
    db.list_collection_names()
else:
    print("Collections doesn't exist yet")

## Create new collection to input into database

In [340]:
result_1999_2018_cleaned[:3]

SEQN  SMD410       Year
1999-2000 0   1.0     2.0  1999-2000
          1   2.0     2.0  1999-2000
          2   3.0     1.0  1999-2000

In [341]:
#Set SEQN as _id (Primary Key)
result_1999_2018_cleaned.rename(columns={'SEQN':'_id'}, inplace=True)

In [342]:
#Dataframe to dictionary
smqfam_dict = result_1999_2018_cleaned.to_dict(orient='records')

In [343]:
smqfam_dict[0]

{'_id': 1.0, 'SMD410': 2.0, 'Year': '1999-2000'}

In [344]:
#Insert collection 
smqfam.insert_many(smqfam_dict)

In [345]:
#View collections
db.list_collection_names()

['hiq',
 'alq',
 'drxtot',
 'smqfam',
 'smq',
 'bpq',
 'bmx',
 'huq',
 'bpx',
 'mcq_h',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'demo',
 'diq',
 'mcq_c']

In [346]:
# for s in smqfam.find():
#     print(s)